In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 2080 Ti'

In [3]:
# import wandb

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="unigram-bert-prothom-alo-epoch6",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 2e-5,
#     "weight_decay" : 0.01,
#     "architecture": "unigram-bert-base",
#     "dataset": "prothom_alo",
#     "epochs": 6,
#     "batch size": 64,
#     }
# )


In [4]:
from datasets import load_dataset

prothom_alo_dataset = load_dataset("text", data_files="../datasets/Bangla Prothom Alo.txt", split="train")

Found cached dataset text (/home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


In [5]:
prothom_alo_dataset.set_format("pandas")

In [6]:
prothom_alo_dataset.set_format("pandas")

In [7]:
prothom_alo_df = prothom_alo_dataset[:]

In [8]:
prothom_alo_df_lens = prothom_alo_df['text'].str.len()

In [9]:
max(prothom_alo_df_lens)

219

In [10]:
(prothom_alo_df=='').sum()

text    1350000
dtype: int64

In [11]:
# pretraining_df_lens = pretraining_df['text'].str.len()
count = prothom_alo_df['text'].str.split().apply(len).value_counts()

In [12]:
count.index = count.index.astype(str) + ' words:'
count.sort_index(inplace=True)

In [13]:
count

text
0 words:     1350000
10 words:     144211
11 words:     136289
12 words:     125121
13 words:     114017
14 words:     100848
15 words:      87522
4 words:       39962
5 words:       75678
6 words:      106133
7 words:      130050
8 words:      143045
9 words:      147124
Name: count, dtype: int64

In [14]:
prothom_alo_dataset.reset_format()

In [15]:
prothom_alo_dataset

Dataset({
    features: ['text'],
    num_rows: 2700000
})

In [16]:
prothom_alo_dataset = prothom_alo_dataset.filter(lambda x: x["text"]!="")

Loading cached processed dataset at /home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-c308135649b65eaf.arrow


In [17]:
prothom_alo_dataset

Dataset({
    features: ['text'],
    num_rows: 1350000
})

In [18]:
prothom_alo_dataset[100:200]

{'text': ['বাংলাদেশে রাজনৈতিক দলের সংখ্যা কত তা কেউ জানেন না ।',
  'এ সময় আইনশৃঙ্খলাসহ সবকিছু নির্বাচন কমিশনের কাছে থাকবে ।',
  'রাজনৈতিক দলের নিবন্ধনবর্তমান 1111111111 অনুচ্ছেদটি নিম্নরুপ 1111111111 ।',
  'ছবির নাম দাগ ।',
  'কেন তিনি এ বেনজির হত্যা মামলার তদন্ত করলেন না ?',
  'আমরা শুধু অনুঘটকের দায়িত্ব পালন করি ।',
  'এ কর্মসূচিতে তিনটি হলের প্রায় 1111111111 ছাত্রী অংশ নেন ।',
  'প্রশ্ন আপনার সময়ে তো অনেকেই লিখেছেন ?',
  'হেলথ মাস্টার হেলথ পাল স্বাস্থ্যকথন ও হ্যালোপ্যাথি রয়েছে স্বাস্থ্য ও জীবনযাপন বিভাগে ।',
  'পুলিশ ঘটনার সঙ্গে জড়িত থাকার সন্দেহে দুজনকে আটক করেছে ।',
  'উইকেট পেয়ে ইশান্তের গর্জনরত অভিব্যক্তিটা দেখে ভয় পেতেই পারত অস্ট্রেলিয়া ।',
  'এ সময় তাঁরা ওই বাঁধগুলোর 1111111111 শতাংশ কাজ হয়েছে বলে দেখতে পান ।',
  'ঠাঁই মেলে রোহিঙ্গা আশ্রয়শিবিরে ।',
  'অস্কার অনুষ্ঠানের আর বেশি দিন বাকি নেই ।',
  'এক প্রশ্নের জবাবে তিনি বলেন এখন আর ঝুঁকি নেই ।',
  'মার্কিন মুলুকে বাঙালি অভিবাসীদের জীবনচিত্র অত্যন্ত হৃদয়গ্রাহীভাবে তুলে ধরেছেন নাসরিন চৌধুরী ।',
  'সপ্তাহে তিন দিন এখানে এসে অফিস ক

In [19]:
def get_training_corpus():
    for i in range(0, len(prothom_alo_dataset), 1000):
        yield prothom_alo_dataset[i : i+1000]["text"]

In [20]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [21]:
tokenizer = Tokenizer(models.Unigram())

In [22]:
from tokenizers import Regex

tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKC(),
#         normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

In [19]:
print(tokenizer.normalizer.normalize_str('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।'))

রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।


In [20]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)

In [21]:
# tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

In [22]:
tokenizer.pre_tokenizer.pre_tokenize_str('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')

[('▁রিয়াল', (0, 5)),
 ('▁ভাবতে', (5, 11)),
 ('▁পারে', (11, 16)),
 ('▁তারা', (16, 21)),
 ('▁আসলে', (21, 26)),
 ('▁1111111111', (26, 37)),
 ('▁পয়েন্টে', (37, 45)),
 ('▁এগিয়ে', (45, 51)),
 ('▁।', (51, 53))]

In [23]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.UnigramTrainer(
    vocab_size=30522, special_tokens=special_tokens, unk_token="[UNK]"
)

In [24]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [25]:
encoding = tokenizer.encode('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')
print(encoding.tokens)

['▁রিয়াল', '▁ভাবতে', '▁পারে', '▁তারা', '▁আসলে', '▁', '1', '111111111', '▁পয়েন্টে', '▁এগিয়ে', '▁', '।']


In [26]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [27]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [28]:
encoding = tokenizer.encode('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')
print(encoding.tokens)

['[CLS]', '▁রিয়াল', '▁ভাবতে', '▁পারে', '▁তারা', '▁আসলে', '▁', '1', '111111111', '▁পয়েন্টে', '▁এগিয়ে', '▁', '।', '[SEP]']


In [29]:
encoding = tokenizer.encode("এ ছাড়া শিক্ষাপ্রতিষ্ঠানেও চলবে প্রচারণা ।","সহযোগিতা করছে তথ্য ও যোগাযোগপ্রযুক্তি আইসিটি বিভাগ ।")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', '▁এ', '▁ছাড়া', '▁', 'শিক্ষাপ্রতিষ্ঠান', 'ে', 'ও', '▁চলবে', '▁প্রচারণা', '▁', '।', '[SEP]', '▁সহযোগিতা', '▁করছে', '▁তথ্য', '▁', 'ও', '▁', 'যোগাযোগপ্রযুক্তি', '▁আইসিটি', '▁বিভাগ', '▁', '।', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [30]:
tokenizer.decoder = decoders.Metaspace()

In [31]:
tokenizer.decode(encoding.ids)

'এ ছাড়া শিক্ষাপ্রতিষ্ঠানেও চলবে প্রচারণা । সহযোগিতা করছে তথ্য ও যোগাযোগপ্রযুক্তি আইসিটি বিভাগ ।'

In [32]:
tokenizer.enable_truncation(max_length=512)

In [33]:
tokenizer.save("unigram_tokenizer_prothom_alo.json")

In [62]:
# tokenizer = Tokenizer.from_file("unigram_tokenizer_prothom_alo.json")

In [21]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
    #tokenizer_object=tokenizer,
    tokenizer_file="unigram_tokenizer_prothom_alo.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    return_special_tokens_mask = True,
    model_max_length = 512,
)

In [20]:
tokenizer.save_pretrained("unigram_tokenizers_prothom_alo")

('unigram_tokenizers_prothom_alo/tokenizer_config.json',
 'unigram_tokenizers_prothom_alo/special_tokens_map.json',
 'unigram_tokenizers_prothom_alo/tokenizer.json')

In [24]:
# from transformers import PreTrainedTokenizerFast

# tokenizer = PreTrainedTokenizerFast.from_pretrained("unigram_tokenizers_prothom_alo")

In [22]:
from transformers import BertConfig, BertForMaskedLM

# Building the config
config = BertConfig()

print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [23]:
#test tokenizer
# tokenizer.tokenize("শিক্ষাপ্রতিষ্ঠানেও")
# tokenizer.tokenize("পাচজনকে")
# tokenizer.tokenize("অংশগুলি")

In [24]:
unigram_bert_config = BertConfig(pad_token_id=tokenizer.pad_token_id)
print(unigram_bert_config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [24]:
unigram_bert_config.save_pretrained(save_directory="configs/unigram_bert_config/")

In [25]:
unigram_bert_config = BertConfig.from_pretrained("configs/unigram_bert_config/config.json")
# Building the model from the config
# Model is randomly initialized
model = BertForMaskedLM(unigram_bert_config)

In [26]:
unigram_bert_config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [26]:
total_params = 0
for p in model.parameters():
    if len(p.shape) == 2:
        total_params += p.shape[0] * p.shape[1]
        
print(f"Total parameters: {total_params:,}")

Total parameters: 109,360,128


In [27]:
# text = 'পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'
text = 'পরে সেখানে সংক্ষিপ্ত সমাবেশ [MASK] হয় ।'

In [28]:
tokenizer.tokenize(text)

['▁পরে', '▁সেখানে', '▁সংক্ষিপ্ত', '▁সমাবেশ', '▁', '[MASK]', '▁হয়', '▁', '।']

In [29]:
import torch 

inputs = tokenizer(text, return_tensors="pt")
# inputs.to("cuda")

token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ মৃণ্ময় হয় ।'
'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ পাথরবোঝা হয় ।'
'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ বিদ্যালয়গুলো হয় ।'
'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ সচেতন হয় ।'
'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ লেখক হয় ।'


In [30]:
def tokenize_function(examples):
    result = tokenizer(examples["text"], padding="max_length", max_length=80, truncation=True)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = prothom_alo_dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

Loading cached processed dataset at /home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-f15318c8addee905.arrow


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 1350000
})

In [31]:
tokenized_datasets.remove_columns("token_type_ids")

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 1350000
})

In [32]:
# temp = tokenized_datasets.filter(lambda x:x if 1 in x["input_ids"] else None)

In [33]:
tokenized_datasets[0]

{'input_ids': [2,
  1434,
  1843,
  80,
  111,
  574,
  5,
  7,
  8,
  4444,
  341,
  5,
  6,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [34]:
def group_texts(examples):
    # Create a new labels column
    examples["labels"] = examples["input_ids"].copy()
    return examples

In [35]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/1350000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 1350000
})

In [37]:
tokenizer.decode(lm_datasets[1]["input_ids"])

'[CLS] পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [38]:
tokenizer.decode(lm_datasets[1]["labels"])

'[CLS] পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [39]:
tokenizer.pad_token_id

1

In [40]:
import collections
import numpy as np
np.random
from transformers import default_data_collator

wwm_probability = 0.15


def bangla_data_collator(features):
    for feature in features:
#         word_ids = feature.pop("word_ids")

#         # Create a map between words and corresponding token indices
#         mapping = collections.defaultdict(list)
#         current_word_index = -1
#         current_word = None
#         for idx, word_id in enumerate(word_ids):
#             if word_id is not None:
#                 if word_id != current_word:
#                     current_word = word_id
#                     current_word_index += 1
#                 mapping[current_word_index].append(idx)

        # Randomly mask words
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        mask = np.random.binomial(1, wwm_probability, (len(input_ids),))
        special_tokens =  [tokenizer.unk_token_id, tokenizer.pad_token_id, tokenizer.cls_token_id, \
                           tokenizer.sep_token_id, tokenizer.mask_token_id]
        
        new_labels = [-100] * len(labels)
        for idx in np.where(mask)[0]:
#             word_id = word_id.item()
#             print(word_id)
#             for idx in mapping[word_id]:
#             if word_ids[idx] is not None:
            if input_ids[idx] not in special_tokens:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
            feature["labels"] = new_labels
        
    return default_data_collator(features)

In [41]:
from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
lm_datasets = lm_datasets.remove_columns(["word_ids"])
data_collator = bangla_data_collator

In [42]:
samples = [lm_datasets[i] for i in range(3)]
# for sample in samples:
#     _ = sample.pop("word_ids")

for chunk in bangla_data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] রিয়াল ভাবতে পারে[MASK] আসলে 1111111111[MASK] এগিয়ে ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

'>>> [CLS] পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত[MASK] ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

'>>> [CLS] বেঁচে থাকার[MASK] ভাবিনি বলছিলেন নূর মোহাম্মদ ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]

In [43]:
prothom_alo_dataset[:3]

{'text': ['রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।',
  'পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।',
  'বেঁচে থাকার কথা ভাবিনি বলছিলেন নূর মোহাম্মদ ।']}

In [44]:
samples = [lm_datasets[i] for i in range(1)]

chunk = data_collator(samples)
print(chunk["input_ids"])
print(chunk["labels"])

tensor([[   2, 1434, 1843,    4,  111,  574,    5,    7,    4, 4444,  341,    5,
            6,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]])
tensor([[-100, -100, -100,   80, -100, -100, -100, -100,    8, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100,

In [45]:
# train_size = 10_000
# test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets.train_test_split(
    train_size=0.8, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1080000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 270000
    })
})

In [46]:
for idx, sample in enumerate(downsampled_dataset["train"]["input_ids"][:3]):
    print(f"'>>> Article {idx} length: {len(sample)}'")

'>>> Article 0 length: 80'
'>>> Article 1 length: 80'
'>>> Article 2 length: 80'


In [51]:
tokenizer.decode(downsampled_dataset["test"]["input_ids"][0])

'[CLS] যত দ্রুত সম্ভব এই ঘুষ দুর্নীতি দমন করতে হবে ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [67]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [68]:
# from __future__ import division

In [69]:
from transformers import TrainingArguments

batch_size = 64
eval_batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size


training_args = TrainingArguments(
    num_train_epochs = 3,
    report_to = None,
    output_dir="models/unigram/bert-base-pretrained-prothom-alo",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    #push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
    load_best_model_at_end=True,
    save_strategy = "epoch",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [70]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using amp half precision backend


In [71]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 270000
  Batch size = 64


>>> Perplexity: 25.81


In [60]:
trainer.train()

***** Running training *****
  Num examples = 1080000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 50625


Epoch,Training Loss,Validation Loss
1,3.075400,3.390895
2,3.010000,3.308410
3,3.006700,3.252617


***** Running Evaluation *****
  Num examples = 270000
  Batch size = 64
Saving model checkpoint to models/unigram/bert-base-pretrained-prothom-alo/checkpoint-16875
Configuration saved in models/unigram/bert-base-pretrained-prothom-alo/checkpoint-16875/config.json
Model weights saved in models/unigram/bert-base-pretrained-prothom-alo/checkpoint-16875/pytorch_model.bin
tokenizer config file saved in models/unigram/bert-base-pretrained-prothom-alo/checkpoint-16875/tokenizer_config.json
Special tokens file saved in models/unigram/bert-base-pretrained-prothom-alo/checkpoint-16875/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 270000
  Batch size = 64
Saving model checkpoint to models/unigram/bert-base-pretrained-prothom-alo/checkpoint-33750
Configuration saved in models/unigram/bert-base-pretrained-prothom-alo/checkpoint-33750/config.json
Model weights saved in models/unigram/bert-base-pretrained-prothom-alo/checkpoint-33750/pytorch_model.bin
tokenizer config file 

TrainOutput(global_step=50625, training_loss=3.0306976080246915, metrics={'train_runtime': 9800.6714, 'train_samples_per_second': 330.59, 'train_steps_per_second': 5.165, 'total_flos': 1.3324743648e+17, 'train_loss': 3.0306976080246915, 'epoch': 3.0})

In [61]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 270000
  Batch size = 64


>>> Perplexity: 26.05


In [62]:
trainer.save_model()

Saving model checkpoint to models/unigram/bert-base-pretrained-prothom-alo
Configuration saved in models/unigram/bert-base-pretrained-prothom-alo/config.json
Model weights saved in models/unigram/bert-base-pretrained-prothom-alo/pytorch_model.bin
tokenizer config file saved in models/unigram/bert-base-pretrained-prothom-alo/tokenizer_config.json
Special tokens file saved in models/unigram/bert-base-pretrained-prothom-alo/special_tokens_map.json


In [128]:
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

eval/loss,▇▅▂█▇█▅▁█▇█
eval/runtime,▇▄█▂▁▂▄▄▁▁▃
eval/samples_per_second,▂▄▁▆█▇▅▄█▇▆
eval/steps_per_second,▂▄▁▆█▇▅▄█▇▆
train/epoch,█▁▁▅▅████▁▁▅▅████
train/global_step,█▁▃▃▆▆████▁▃▃▆▆████
train/learning_rate,█▄▁█▄▁
train/loss,▅▇█▁▅▇
train/total_flos,▁▁
train/train_loss,█▁
train/train_runtime,█▁


In [65]:
model = BertForMaskedLM.from_pretrained("models/unigram/bert-base-pretrained-prothom-alo")

loading configuration file models/unigram/bert-base-pretrained-prothom-alo/config.json
Model config BertConfig {
  "_name_or_path": "models/unigram/bert-base-pretrained-prothom-alo/checkpoint-101250",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.16.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file models/unigram/bert-base-pretrained-prothom-alo/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were init

In [66]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("models/unigram/bert-base-pretrained-prothom-alo")

Didn't find file models/unigram/bert-base-pretrained-prothom-alo/added_tokens.json. We won't load it.
loading file None
loading file models/unigram/bert-base-pretrained-prothom-alo/special_tokens_map.json
loading file models/unigram/bert-base-pretrained-prothom-alo/tokenizer_config.json
loading file models/unigram/bert-base-pretrained-prothom-alo/tokenizer.json


In [186]:
import torch
torch.cuda.empty_cache()